In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/text_classification/dot/word2vec-ernie

/content/drive/MyDrive/text_classification/dot/word2vec-ernie


In [4]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import jieba

In [51]:
## ernie ##
# 读取数据
data = pd.read_excel('./data.xlsx')
sentences = data['review'].tolist()

def txt_cut(s):
    res = [w for w in jieba.lcut(s) if w.strip()]
    return " ".join(res)

# 对文本进行分词
tokenized_text = []
for i in sentences:
    new = txt_cut(i).split(' ')
    tokenized_text.extend(new)

# 初始化ERNIE tokenizer 和 model
tokenizer = AutoTokenizer.from_pretrained('nghuyong/ernie-1.0')
model = AutoModel.from_pretrained('nghuyong/ernie-1.0')

# 用于存储单词和词向量的字典
word_vectors_dict = {}

# 使用jieba分词并处理每个单词
for word in tokenized_text:
    # 使用ERNIE tokenizer 对单词进行编码
    inputs = tokenizer(word, return_tensors='pt', truncation=True, padding=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # 获取单词的词向量
    word_vector = outputs.last_hidden_state.squeeze(0).numpy()

    # 存储单词和对应词向量
    word_vectors_dict[word] = word_vector

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
import pandas as pd
from gensim.models import Word2Vec
import jieba

In [52]:
## word2vec ##
# 初始化Word2Vec模型
model_w = Word2Vec(vector_size=100, window=5, min_count=1)

# 构建词汇表
model_w.build_vocab([tokenized_text])

# 训练模型
model_w.train([tokenized_text], total_examples=model_w.corpus_count, epochs=model_w.epochs)

# 创建一个空的DataFrame来保存词汇和对应的词向量
word_vector_df = pd.DataFrame()

for wd in model_w.wv.index_to_key:
    # 将词向量转化为Series，然后添加到DataFrame中
    word_vector_series = pd.Series([wd] + list(model_w.wv[wd]))
    word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)

<ipython-input-52-cc1d6ea3e34b>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-52-cc1d6ea3e34b>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-52-cc1d6ea3e34b>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-52-cc1d6ea3e34b>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipy

In [54]:
new_word_vectors = []

## ernie
for word, vector in word_vectors_dict.items():
    word_vector_r = vector[0]
    word_vector_r = word_vector_r.reshape(-1, 1)

    ## word2vec
    word_vector_w = model_w.wv[word]
    word_vector_w = word_vector_w.reshape(-1, 1)

    ## 新词向量
    new_word_vector = np.multiply(word_vector_r, word_vector_w.T)
    new_word_vectors.append((word, new_word_vector))

In [58]:
# 指定保存文件路径
save_path = 'new_word_vectors.txt'

# 保存单词和新词向量到文件
with open(save_path, 'w') as f:
    for word, word_vector in new_word_vectors:
        word_vector_str = ','.join(map(str, word_vector))
        line = f"{word}: {word_vector_str}\n"
        f.write(line)